# Preprocessing

In [27]:
api_key = ""
channel_id = ""

In [28]:
from googleapiclient.discovery import build
youtube = build("youtube", "v3", developerKey=api_key)

In [29]:
def get_channel_videos(channel_id):
    res = youtube.channels().list(id=channel_id,
                                 part="contentDetails").execute()
    playlist_id = res["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]
    
    videos = []
    next_page_token = None
    
    while 1:
        res = youtube.playlistItems().list(playlistId=playlist_id,
                                          part="snippet",
                                          maxResults=50,
                                          pageToken=next_page_token).execute()
        videos += res["items"]
        next_page_token = res.get("nextPageToken")
        
        if next_page_token is None:
            break
    
    return videos

In [30]:
videos = get_channel_videos(channel_id)

In [31]:
res = youtube.videos().list(id=videos[0]["snippet"]["resourceId"]["videoId"],
                           part="statistics").execute()

In [32]:
video_ids = list(map(lambda x:x['snippet']['resourceId']['videoId'], videos))

In [33]:
def get_videos_stats(video_ids):
    stats = []
    for i in range(0, len(video_ids), 50):
        res = youtube.videos().list(id=','.join(video_ids[i:i+50]),
                                   part='statistics').execute()
        stats += res['items']
    return stats

In [34]:
stats = get_videos_stats(video_ids)

# List Before Dataframe

In [49]:
publishedAt = []
title = []
viewCount = []
likeCount = []
dislikeCount = []
commentCount = []

for i in range(len(videos)):
    publishedAt.append(videos[i]['snippet']['publishedAt'])
    title.append(videos[i]['snippet']['title'])
    viewCount.append(stats[i]['statistics']['viewCount'])
    likeCount.append(stats[i]['statistics']['likeCount'])
    dislikeCount.append(stats[i]['statistics']['dislikeCount'])
    commentCount.append(stats[i]['statistics']['commentCount'])

# Dataframe
공개일, 제목, 조회수, 좋아요수, 싫어요수, 댓글수

In [50]:
import pandas as pd
names = ['publishedAt', 'title', 'viewCount', 'likeCount', 'dislikeCount', 'commentCount']
df = pd.DataFrame(data=list(zip(publishedAt, title, viewCount, likeCount, dislikeCount, commentCount)),
                 columns=names)

# int로 변환

In [51]:
df['viewCount'] = df['viewCount'].astype(int)
df['likeCount'] = df['likeCount'].astype(int)
df['dislikeCount'] = df['dislikeCount'].astype(int)
df['commentCount'] = df['commentCount'].astype(int)

# 라이브 영상 Drop하기

In [52]:
import re
def reg_live(word):
    if re.compile('라이브').search(word):
        return True
    else:
        return False

In [53]:
df['live'] = df['title'].apply(reg_live)

In [54]:
df = df.loc[(df['live'] == False)]
len(df)

2582

# 과거 절반 데이터 Drop하기

In [55]:
df = df[:int(len(df)/2)]
len(df)

1291

# 최근 데이터 Drop하기
공개된지 몇 시간 되지 않아 아직 조회수가 채워지지 않았기 때문에 같은 기준으로 분석할 수 없다.

In [56]:
df = df[5:]
len(df)

1286

# 필요한 column 빼고 전부 Drop하기

In [57]:
df = df[['title', 'viewCount']]

# 전처리 완료

In [58]:
df.head()

,title,viewCount
5,에이치엘비 허위공시 워딩이 자꾸 나오는데 진짜 허위공시는 이런 것이다!!,6634
6,에이치엘비 추가 무상증자 가능성과 엘생의 지분법 평가내역!!,16849
7,에이치엘비와 어딘가 순서가 거꾸로된 TS인베스트먼트의 무증후 대주주매도!,7471
8,에이치엘비 계열회사 넥스트사이언스 임원의 스톡옵션 단타 시도?!,5987
9,에이치엘비 4월 미국암학회는 몸 풀기고 6월 ASCO는 본게임이다!,9769


# 분석 시작 

# 제목에 '에이치엘비'가 있는가?

In [59]:
df['에이치엘비'] = df['title'].str.contains('에이치엘비')

# 제목에 'FDA'가 있는가?
대/소문자 모두 포함하기 위해 re.I를 사용한다.

In [60]:
import re
def reg_func(word):
    if re.compile('FDA', re.I).search(word):
        return True
    else:
        return False

In [61]:
df['fda'] = df['title'].apply(reg_func)

# 제목에 한글만 사용했는가?
True : 한글만 사용  
False : 영어도 사용

In [63]:
import re
def reg_func(word):
    if re.compile('[a-zA-Z]+').search(word):
        return False
    else:
        return True

In [64]:
df['hangul'] = df['title'].apply(reg_func)

# 제목에 '넥스트사이언스'가 있는가?
True : 넥스트사이언스 있다.  
False : 넥스트사이언스 없다. (없어야 조회수 높다.)

In [66]:
df['넥스트사이언스'] = df['title'].str.contains('넥스트사이언스')

# 글자수가 40자 이하인가?

In [96]:
df['글자수'] = df['title'].apply(len) <= 40

# 느낌표를 사용했는가?

In [102]:
df['느낌표'] = df['title'].str.contains('!')

In [104]:
df.head()

,title,viewCount,에이치엘비,fda,hangul,넥스트사이언스,글자수,느낌표
5,에이치엘비 허위공시 워딩이 자꾸 나오는데 진짜 허위공시는 이런 것이다!!,6634,True,False,True,False,True,True
6,에이치엘비 추가 무상증자 가능성과 엘생의 지분법 평가내역!!,16849,True,False,True,False,True,True
7,에이치엘비와 어딘가 순서가 거꾸로된 TS인베스트먼트의 무증후 대주주매도!,7471,True,False,False,False,True,True
8,에이치엘비 계열회사 넥스트사이언스 임원의 스톡옵션 단타 시도?!,5987,True,False,True,True,True,True
9,에이치엘비 4월 미국암학회는 몸 풀기고 6월 ASCO는 본게임이다!,9769,True,False,False,False,True,True


# Y Label 만들기
상위 25%는 1로 할당하고  
나머지 75%는 0으로 할당한다.

In [148]:
quantile75 = df['viewCount'].quantile(q=0.75)
quantile75

7722.0

In [118]:
df['y'] = df['viewCount'] >= quantile75

In [121]:
y = df['y']

# X Feature 만들기

In [123]:
X = df[['에이치엘비', 'fda', 'hangul', '넥스트사이언스', '글자수', '느낌표']]

In [124]:
X.head()

,에이치엘비,fda,hangul,넥스트사이언스,글자수,느낌표
5,True,False,True,False,True,True
6,True,False,True,False,True,True
7,True,False,False,False,True,True
8,True,False,True,True,True,True
9,True,False,False,False,True,True


# 머신러닝

In [125]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [126]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression().fit(X_train, y_train)
logreg.score(X_test, y_test)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7236024844720497

# 예측해보기
에이치엘비	fda	hangul	넥스트사이언스	글자수	느낌표

에이치엘비만 제목에 있을 경우 조회수 7,725회(상위 25%)를 넘지 못 한다.

In [147]:
logreg.predict([[1,0,0,0,0,0]])

array([False])

에이치엘비와 한글

In [150]:
logreg.predict([[1,0,1,0,0,0]])

array([False])

에이치엘비와 글자수

In [151]:
logreg.predict([[1,0,0,0,1,0]])

array([False])

에이치엘비와 느낌표

In [152]:
logreg.predict([[1,0,0,0,0,1]])

array([False])

에이치엘비+hangul+느낌표

In [156]:
logreg.predict([[1,0,1,0,0,1]])

array([False])

에이치엘비+hangul+글자수

In [157]:
logreg.predict([[1,0,1,0,1,0]])

array([False])

에이치엘비+hangul+글자수+느낌표

In [158]:
logreg.predict([[1,0,1,0,1,1]])

array([False])

에이치엘비와 fda

In [160]:
logreg.predict([[1,1,0,0,0,0]])

array([ True])

# Results
정확도 72%  
제목에 '에이치엘비'와 'fda' 를 포함한 조합만이 상위 25% 조회수가 나온다고 예측된다.  

# Conclusion
제목에 '에이치엘비'와 'fda'를 포함한다.  
워밍업이었으므로 향후 엄밀한 머신러닝을 적용해본다. confusion matrix와 ROC curve도 구해본다.